Lambda School Data Science, Unit 2: Predictive Modeling

# Applied Modeling, Module 4

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Continue to iterate on your project: data cleaning, exploratory visualization, feature engineering, modeling.
- [ ] Make a Shapley force plot to explain at least 1 individual prediction.
- [ ] Share at least 1 visualization on Slack.

(If you haven't completed an initial model yet for your portfolio project, then do today's assignment using your Tanzania Waterpumps model.)

## Stretch Goals
- [ ] Make Shapley force plots to explain at least 4 individual predictions.
    - If your project is Binary Classification, you can do a True Positive, True Negative, False Positive, False Negative.
    - If your project is Regression, you can do a high prediction with low error, a low prediction with low error, a high prediction with high error, and a low prediction with high error.
- [ ] Use Shapley values to display verbal explanations of individual predictions.
- [ ] Use the SHAP library for other visualization types.

The [SHAP repo](https://github.com/slundberg/shap) has examples for many visualization types, including:

- Force Plot, individual predictions
- Force Plot, multiple predictions
- Dependence Plot
- Summary Plot
- Summary Plot, Bar
- Interaction Values
- Decision Plots

We just did the first type during the lesson. The [Kaggle microcourse](https://www.kaggle.com/dansbecker/advanced-uses-of-shap-values) shows two more. Experiment and see what you can learn!


## Links
- [Kaggle / Dan Becker: Machine Learning Explainability — SHAP Values](https://www.kaggle.com/learn/machine-learning-explainability)
- [Christoph Molnar: Interpretable Machine Learning — Shapley Values](https://christophm.github.io/interpretable-ml-book/shapley.html)
- [SHAP repo](https://github.com/slundberg/shap) & [docs](https://shap.readthedocs.io/en/latest/)

In [60]:
import sys, getpass
a = getpass.getuser()
### example appending of folder
# A
sys.path.append('/home/{}/Code/Github-Repos/BFLibrary/'.format(a))
# B
sys.path.append('/Users/{}/Programs/BFLibrary/'.format(a))
#from BFLibrary import ACompass, BWarp, CToolbox 
import AMap as am
import BCook as bc
import CTools as ct

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')
import pandas as pd
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')
### Select features, scikit-learn pipeline to encode categoricals, impute missing values, and fit a decision tree classifier.
import category_encoders as ce

import xgboost as xgb
from xgboost import XGBClassifier

from scipy.stats import randint, uniform

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

import eli5
from eli5.sklearn import PermutationImportance

### Wrangle function
def wrangle(df):
    """I heard you like docstrings"""
    import numpy as np
    
    ### Easy stuff
    # Copy the dataframe to itself so warnings can shsshshsh
    df = df.copy()
    
    df['latitude'] = df['latitude'].replace(-2e-08,0)
    df['date_recorded'] = pd.to_datetime(df['date_recorded'],infer_datetime_format=True)
    df['year_recorded'] = df['date_recorded'].dt.year
    df['month_recorded'] = df['date_recorded'].dt.month
    df['day_recorded'] = df['date_recorded'].dt.day
    
    # Drop dupes
    df = df.drop(columns=['date_recorded','quantity_group',
                          'scheme_management','quality_group',
                          'waterpoint_type_group','extraction_type_group'])

    # Ryan's Engineer feature: how many years from construction_year to date_recorded
    df['years'] = df['year_recorded'] - df['construction_year']
    df['years_MISSING'] = df['years'].isnull()

    # YEEHAW dataframe return
    return df

### Make target
target = 'status_group'

In [96]:
test.shape, train.shape, sample_submission.shape

((14358, 40), (59400, 41), (14358, 2))

In [70]:
# Assign variables
X_train = wrangle(train)
X_train = X_train.drop(target,axis=1)
y_train = train[target]
X_test = wrangle(test)

In [72]:
print(train.shape, "\n", X_train.shape, y_train.shape, X_test.shape)

(59400, 41) 
 (59400, 39) (59400,) (14358, 39)


In [73]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=10000, stratify=y_train, random_state=4
)
X_train.shape,X_val.shape,y_train.shape,y_val.shape
# Save IDs like Ryans to look up results,
# to compare w/ predicted results
train_id = X_train['id']
val_id = X_val['id']
test_id = X_test['id']

In [74]:
X_train.shape, y_train.shape, X_val.shape, y_train.shape

((49400, 39), (49400,), (10000, 39), (49400,))

In [ ]:
# Okay now we encode, impute and process manually.

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier

processor = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median')
)

X_train_p = processor.fit_transform(X_train)
X_val_p = processor.transform(X_val)

In [80]:
e_set = [(X_train_p, y_train),
         (X_val_p, y_val)]

2


In [86]:
model = XGBClassifier(n_estimators=200, n_jobs=-1)

# Eval metric gives you ROC AUC but for this ain't workin
model.fit(X_train_p, y_train,
          eval_set=e_set,
          early_stopping_rounds=5)

[0]	validation_0-merror:0.29504	validation_1-merror:0.3002
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 5 rounds.
[1]	validation_0-merror:0.29504	validation_1-merror:0.3002
[2]	validation_0-merror:0.29504	validation_1-merror:0.3002
[3]	validation_0-merror:0.293563	validation_1-merror:0.2988
[4]	validation_0-merror:0.294433	validation_1-merror:0.2996
[5]	validation_0-merror:0.294433	validation_1-merror:0.2996
[6]	validation_0-merror:0.293563	validation_1-merror:0.2988
[7]	validation_0-merror:0.293583	validation_1-merror:0.2988
[8]	validation_0-merror:0.293563	validation_1-merror:0.2988
Stopping. Best iteration:
[3]	validation_0-merror:0.293563	validation_1-merror:0.2988



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [98]:
X_test_processed = processor.transform(X_test)
y_pred = model.predict(X_test_processed)
print('Classy:', accuracy_score(y_pred,sample_submission))

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
import shap

row = X_val.iloc[[100]]

explainer = shap.TreeExplainer(model)
row_processed = processor.transform(row)

# How each feature added up to final prediction
shap_values = explainer.shap_values(row_processed)

# omg js
shap.initjs()
shap.force_plot(
    # What we predict by default
    base_value=explainer.expected_value,
    # The nudgers
    shap_values=shap_values,
    # Just the row to use feature names/values
    features=row
)